# Import Python Modules

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression

## Approach
In this notebook, revenue will be estimated with an ensemble of machine learning techniques. Specifically:
* Because there are many predictors, the model space will be managed by
    * PCA to compress variable space and/or
    * Ridge Regression to minimize overfitting by large coefficient values 
* Regression 
* Regression Trees

# Read in Data
Some initial preprocessing was done:
* Replaced blanks with NAs (NaN in df below)
* Moved the reponse variable (revenue) to the first field

Preprocessing done in R ('nuggetsPreprocess.R')
* Changed binary variables to y/n for portability
* Added variables rev1weekAgo - rev4weekAgo and weekOfYear for handling temporal autocorrelation and seasonality

In [2]:
df = pd.read_csv('dataClean.csv')
df2 = pd.read_csv('dataCleanNoOutlier.csv')
df.head()

,revenue,Week,weekno,highseason,holiday,displayimp,displaylag,brandpaid,brandclicks,nonbrandpaid,...,blogsignup,autoenroll,socialperson,promoteposts,learner,rev1weekAgo,rev2weekAgo,rev3weekAgo,rev4weekAgo,weekOfYear
0,178480,2/2/14,1,y,n,987718,983958.0000,31589,4715,105901,...,18,y,y,n,n,NaN,NaN,NaN,NaN,5
1,185390,2/9/14,2,y,n,660136,907388.9167,30428,4540,108652,...,16,y,y,n,n,178480,NaN,NaN,NaN,6
2,178130,2/16/14,3,y,n,604606,852862.4167,30715,4417,119398,...,21,y,y,n,n,185390,178480,NaN,NaN,7
3,185020,2/23/14,4,y,n,560656,851885.3333,31632,4372,134819,...,19,y,y,n,n,178130,185390,178480,NaN,8
4,170950,3/2/14,5,y,n,326112,827164.4167,NaN,4431,115865,...,11,y,y,n,n,185020,178130,185390,178480,9


## Parse Data into Train and Test Sets
Because we are looking at two data sets (with and without "outliers" removed), we want the same data to be in each Test and Training set for direct comparison.

In [3]:
df.shape

(117, 47)

Hold out ~25% or 29 rows of data for the test set.

In [4]:
# Separate df into X (predictors) and y (response)
X = df.ix[:, 1:47]
y = df.ix[:, 0]

# Same for data with outliers removed
X2 = df2.ix[:, 1:47]
y2 = df2.ix[:, 0]

# Now split each into testing and training sets
# Ideally, we would stratify the data, but there are too few data in this
# sample 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.25, random_state = 11)
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size = 0.25, random_state = 11)

In [5]:
# Verify that splitting was the same for both sets
print X_train.Week[:5], '\n', X2_train.Week[:5]
print y_train[:5], '\n', y2_train[:5]

110    3/13/16
28     8/17/14
51     1/25/15
56      3/1/15
39     11/2/14
Name: Week, dtype: object 
110    3/13/16
28     8/17/14
51     1/25/15
56      3/1/15
39     11/2/14
Name: Week, dtype: object
110    166860
28     150160
51     183670
56     182780
39     152870
Name: revenue, dtype: int64 
110    166860
28     150160
51     183670
56     182780
39     152870
Name: revenue, dtype: int64


For now, ignore the outlier-removed data set (X2, y2), and work with X, y

Start with a basic linear model and use its predictions as a baseline score to beat.

In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)

ValueError: could not convert string to float: y